# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
data = pd.read_csv("../weather_data.csv")
data.sort_values('Max Temp',ascending=False)
data.dtypes

City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = data[["Lat", "Lng"]]
humidity = data['Humidity']
max_humidity = max(humidity)

# Plot Heatmap
fig = gmaps.figure(center=(45, -99),zoom_level=1.5)

# Create layer
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity, point_radius=1)

# Add layer
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
## A max temperature lower than 80 degrees but higher than 70.
## Wind speed less than 10 mph.
## Zero cloudiness.
## Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

data_subset = data[(data['Max Temp']<80)&(data['Max Temp']>70)&(data['Wind Speed'] < 10)&(data['Cloudiness']==0)]
data_subset.head()
len(data_subset)

13

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create a dataframe
hotel_df = data_subset.iloc[:,[0,2]]
hotel_df.head()

,City,Country
36,Tessalit,ML
92,Castro,BR
110,Betioky,MG
138,Sao Filipe,CV
150,Los Llanos de Aridane,ES


In [11]:
# url variable store url 
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotel_name = []
latitude = []
longitude = []

for lat, lng, city in zip(data_subset.iloc[:, 5], data_subset.iloc[:, 6], data_subset.iloc[:, 0]):

    # get method of requests module 
    # return response object
    try:
        target = (f"{url}location={lat},{lng}&radius=5000&type=lodging&key={g_key}")
        r = requests.get(target).json()
        hotel_name.append(r['results'][0]['name'])
        latitude.append(r['results'][0]['geometry']['location']['lat'])
        longitude.append(r['results'][0]['geometry']['location']['lng'])
        print(f"Hotel found!")
    except:
        hotel_name.append("")
        latitude.append("")
        longitude.append("")
        print(f"Hotel in {city} not found!")

# adding hotel name to dataframe
hotel_df['Hotel Name'] = hotel_name
hotel_df['Hotel Latitude'] = latitude
hotel_df['Hotel Longitude'] = longitude
hotel_df.head()

Hotel in Tessalit not found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel in Kalakamati not found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel in Doka not found!


,City,Country,Hotel Name,Hotel Latitude,Hotel Longitude
36,Tessalit,ML,,,
92,Castro,BR,CHACARA BAILLY,-24.7807,-49.9868
110,Betioky,MG,Bungalows Acacias Hotel,-23.7299,44.3816
138,Sao Filipe,CV,Tortuga B&B,14.8807,-24.493
150,Los Llanos de Aridane,ES,Valle Aridane,28.6582,-17.9161


In [18]:
# clean dataframe; drop city with no hotel
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
print(f"Number of cities with hotel = {len(hotel_df)}")
hotel_df.head()

Number of cities with hotel = 10


,City,Country,Hotel Name,Hotel Latitude,Hotel Longitude
92,Castro,BR,CHACARA BAILLY,-24.7807,-49.9868
110,Betioky,MG,Bungalows Acacias Hotel,-23.7299,44.3816
138,Sao Filipe,CV,Tortuga B&B,14.8807,-24.493
150,Los Llanos de Aridane,ES,Valle Aridane,28.6582,-17.9161
185,Itarema,BR,Windvillage,-2.92198,-39.9142


In [19]:
# Create a list containing coordinates
coords = list(zip(hotel_df['Hotel Latitude'], hotel_df['Hotel Longitude']))

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Latitude", "Hotel Longitude"]]

In [21]:
# Add marker layer ontop of heat map

text_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(text_layer)
fig

Figure(layout=FigureLayout(height='420px'))